In [ ]:
import pandas as pd
import numpy as np
import random
random.seed(100)

In [ ]:
train=pd.read_csv("../input/tabular-playground-series-apr-2021/train.csv")
test=pd.read_csv("../input/tabular-playground-series-apr-2021/test.csv")
sample=pd.read_csv("../input/tabular-playground-series-apr-2021/sample_submission.csv")

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.drop(columns=["PassengerId","Name"],inplace=True)
test.drop(columns=["PassengerId","Name"],inplace=True)

In [ ]:
train.drop(columns=[],inplace=True)

In [ ]:
train["Age"].fillna(train["Age"].median(),inplace=True)
train["Fare"].fillna(train["Fare"].median(),inplace=True)
test["Age"].fillna(test["Age"].median(),inplace=True)
test["Fare"].fillna(test["Fare"].median(),inplace=True)

In [ ]:
train

In [ ]:
train.dtypes

In [ ]:
## Extract ticket category
def get_category(x):
    try:
       try:
           if int(x.split()[0]):
                return np.NaN
       except:
           return x.split()[0]
    except:
        return np.NaN

In [ ]:
train["Ticket"]=train["Ticket"].apply(lambda x: get_category(x))
test["Ticket"]=test["Ticket"].apply(lambda x: get_category(x))

In [ ]:
def cabin_class(x):
    try:
        return x[0]
    except:
        return np.NaN

In [ ]:
train["Cabin"]=train["Cabin"].apply(lambda x : cabin_class(x))
test["Cabin"]=test["Cabin"].apply(lambda x : cabin_class(x))

In [ ]:
train.isnull().mean()*100

In [ ]:
train["Cabin"].fillna("Missing",inplace=True)
test["Cabin"].fillna("Missing",inplace=True)
train["Ticket"].fillna("Missing",inplace=True)
test["Ticket"].fillna("Missing",inplace=True)

In [ ]:
train["Embarked"].fillna(train["Embarked"].mode()[0],inplace=True)
test["Embarked"].fillna(test["Embarked"].mode()[0],inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
train["Sex"]=le.fit_transform(train["Sex"])
test["Sex"]=le.transform(test["Sex"])

In [ ]:
le=LabelEncoder()
train["Ticket"]=le.fit_transform(train["Ticket"])
test["Ticket"]=le.transform(test["Ticket"])

In [ ]:
le=LabelEncoder()
train["Cabin"]=le.fit_transform(train["Cabin"])
test["Cabin"]=le.transform(test["Cabin"])

In [ ]:
le=LabelEncoder()
train["Embarked"]=le.fit_transform(train["Embarked"])
test["Embarked"]=le.transform(test["Embarked"])

In [ ]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y=train_test_split(train.drop(columns=["Survived"]),train["Survived"],test_size=0.2,random_state=100)

In [ ]:
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [ ]:
clf=CatBoostClassifier(random_state=100)
clf.fit(train_X,train_y)

In [ ]:
pred=clf.predict(test_X)
print(accuracy_score(test_y,pred))

In [ ]:
## Let's install scikit-learn 0.22.0 as it is compatible with skopt
!pip install   scikit-learn==0.22.0

In [ ]:

# Skopt functions
from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, VerboseCallback, DeltaXStopper
from skopt.space import Real, Categorical, Integer

In [ ]:
clf = CatBoostClassifier(thread_count=2,
                         loss_function='Logloss',
                        
                         od_type = 'Iter',
                         verbose= False
                        )

In [ ]:
search_spaces = {'iterations': Integer(10, 1000),
                 'depth': Integer(1, 8),
                 'learning_rate': Real(0.01, 1.0, 'log-uniform'),
                 'random_strength': Real(1e-9, 10, 'log-uniform'),
                 'bagging_temperature': Real(0.0, 1.0),
                 'border_count': Integer(1, 255),
                 'l2_leaf_reg': Integer(2, 30),
                 'scale_pos_weight':Real(0.01, 1.0, 'uniform')}

In [ ]:
opt = BayesSearchCV(clf,
                    search_spaces,
                    verbose=2,
                    scoring="accuracy",
                    cv=5,
                    n_iter=100,
                    n_jobs=-1,  
                    return_train_score=False,
                    refit=True,
                    optimizer_kwargs={'base_estimator':'GP'},
                    random_state=42)

In [ ]:
# opt.fit(train_X,train_y)

In [ ]:
# opt.best_params_

In [ ]:
## Best model
tuned_clf=CatBoostClassifier(bagging_temperature=0.20401679944445256,border_count=215,depth=4,iterations=936,l2_leaf_reg=9,learning_rate=0.013872642181616959,
                            random_strength=6.418759045093663e-05,
                           scale_pos_weight=  0.9276818238164333,thread_count=2,
                         loss_function='Logloss',
                        
                         od_type = 'Iter',
                         verbose= False)

In [ ]:
tuned_clf.fit(train_X,train_y)

In [ ]:
pred=tuned_clf.predict(test_X)
print(accuracy_score(test_y,pred))

In [ ]:
pd.DataFrame(tuned_clf.feature_importances_,index=train.drop(columns=["Survived"]).columns).sort_values(0).plot(kind="barh",figsize=(20,10))

In [ ]:
sample["Survived"]=tuned_clf.predict(test)

In [ ]:
sample.to_csv("testing_model_best_new.csv",index=0)